
 	Creator Name:   Sara Baradaran, Mahdi Heidari, Zahra Khoramian 		
 	Create Date:    Aug 2020 				
 	Module Name:    Client.py 				
 	Project Name:   Secure File Sharing 	

In [1]:
import os
import socket
import hashlib
import base64
import random
from Crypto.Util import Counter
from Crypto import Random
import string
import json
from datetime import datetime
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes,serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from Crypto.Cipher import AES
from captcha.image import ImageCaptcha

In [2]:

AouthCode = '0'

def connection_setup():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    if s == 0:
        print ('ERROR: Unable to create client socket')
        return None
    server_ip = socket.gethostname()
    server_port = 8500
    s.connect((server_ip, server_port))
    print("Connecting to server ... ")
    return s



def send_command(s, key, command): # Returns 1 if successful, otherwise 0
    cipher_text = encrypt(command, key)
    if cipher_text != None:
        s.send(cipher_text.encode('utf-8'))
        return 1
    print('ERROR: Can not encrypt request to send.')
    return 0

def recieve_replay(s, key): # Returns reply if successful, otherwise None
    replay = s.recv(500)
    replay = replay.decode('utf-8')
    plain_text = decrypt(replay, key)
    #print(plain_text)
    if plain_text != None:
        return plain_text
    print('ERROR: Can not decrypt response.')
    return None

In [3]:

def key_exchange(client_sock): # makes a private key, sends and receives public keys, derives the secret key and returns session key

    try:
        backend = default_backend()
        client_pr = ec.generate_private_key(ec.SECP256R1(), backend)
        client_pub = client_pr.public_key()
        client_sock.send(client_pub.public_bytes(serialization.Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo))
        server_rcv_pub = client_sock.recv(200) # server_rcv_pub is server received public key in bytes
        server_pub = serialization.load_pem_public_key(server_rcv_pub, backend)# server_pub is server  public key in curve object
        shared_data = client_pr.exchange(ec.ECDH(), server_pub)
        secret_key = HKDF(hashes.SHA256(), 32, None, b'Key Exchange', backend).derive(shared_data)
        session_key = secret_key[-16:] # to choose the last 128 bit of secret key
        print('Key exchanged successfully.')
        return session_key
    except:
        print('ERROR: Server blocked the connection because no session key has been set.\n')
        return None

def encrypt(plain_text, key): # returns a json containing ciphertext and nonce for CTR mode
    
    nonce1 = Random.get_random_bytes(8)
    countf = Counter.new(64, nonce1)
    cipher = AES.new(key, AES.MODE_CTR, counter=countf)
    cipher_text_bytes = cipher.encrypt(plain_text.encode('utf-8'))
    nonce = base64.b64encode(nonce1).decode('utf-8')
    cipher_text = base64.b64encode(cipher_text_bytes).decode('utf-8')
    result = json.dumps({'nonce':nonce, 'ciphertext':cipher_text})
    return result

def decrypt(data, key):

    try:
        b64 = json.loads(data)
        nonce = base64.b64decode(b64['nonce'].encode('utf-8'))
        cipher_text = base64.b64decode(b64['ciphertext'])
        countf = Counter.new(64, nonce)
        cipher = AES.new(key, AES.MODE_CTR, counter=countf)
        plain_text = cipher.decrypt(cipher_text)
        return plain_text.decode('utf-8')
    except ValueError:
        return None


In [4]:


def proccess_command(s, key, command):
    global AouthCode
    cmd = command.split()    
    if cmd[0] == "register" and len(cmd) == 5:
        if send_command(s, key, command) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split("\n")
        if len(replay) > 1:
            content = replay[1]
        replay = replay[0]
        if replay[1] == "register" and replay[2] == cmd[1]:
            if replay[0] == "1":
                print("Registered successfully")
                return 1  # Registered successfully
            elif replay[0] == "0":
                print("ERROR: Registeration failed try again") #systematic error
                return 0 # Registeration failed
            elif replay[0] == "-2":
                print("ERROR: This username already exists. Please select another one.")
                return 0
            elif replay[0] == "-3":
                print("ERROR: Password is not strong enough.({})".format(content))
                return 0

    elif cmd[0] == "login" and len(cmd) == 3:
        if send_command(s, key, command) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split("\n")
        content = replay[1]
        AouthCode = replay[2]
        replay = replay[0].split()
        #print('get AouthCode',AouthCode)
        if replay[1] == "login" and replay[2] == cmd[1]:
            if replay[0] == "0":
                print("ERROR: Login was not successfull try again")
                return 0
            elif replay[0] == "1":
                print("Logged in successfully")
                return 1
            elif replay[0] == "-1":
                print("ERROR: Username and password do not match. Please try again")
                return 0
            elif replay[0] == "-4":
                print("ERROR: Your Account is ban for {} minute. try again later".format(content))
                return 0

In [5]:
   
def after_login(s, key, command):
    global AouthCode
    content = ''
    replay = ''
    data = command.split("\n",1)
    cmd = data[0].split()
    if len(data) > 1:
        content = data[1] # to enable writing for more than a line
    
    if cmd[0] == "grant" and len(cmd) == 4:  
        if send_command(s, key, command + '\n' + AouthCode) == 0:
            return 0
        replay = recieve_replay(s, key)
        if replay == None:
            return 0
        replay = replay.split()
        if replay[1] == "grant" and replay[2] == cmd[1] and replay[3] == cmd[2]:
            if replay[0] == "1":
                print("Grant access was successfull")

            elif replay[0] == "0": 
                print("ERROR: Grant access was not successfull(permission denied)")
            
            if replay[0] == "-1":
                print("ERROR: Authentication failing")

            elif replay[0] == "-2":  
                print("ERROR: There is no such file")
                 
    if cmd[0] == "revoce" and len(cmd) == 4:  
        if send_command(s, key, command + '\n' + AouthCode) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split()
        if replay[1] == "revoce" and replay[2] == cmd[1] and replay[3] == cmd[2]:
            if replay[0] == "1":
                print("Revoce access was successfull")

            elif replay[0] == "0": 
                print("ERROR: Revoce access was not successfull(permission denied)")
            
            if replay[0] == "-1":
                print("ERROR: Authentication failing")

            elif replay[0] == "-2":  
                print("ERROR: There is no such file")

    if cmd[0] == "write" and len(cmd) == 2:  
        if send_command(s, key, cmd[0]+ ' ' + cmd[1] + '\n' + content + '\n' + AouthCode) == 0:
            return 0    
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split()
        if replay[3] == "write" and replay[4] == cmd[1]:

            if replay[0] == "1":
                print("Writing in file was successful")

            elif replay[0] == "0" and replay[1] == "1" and replay[2] == "1":   
                print("ERROR: Writing in file was not successfull try again")

            else:
                print("ERROR: Writing in file was not successfull : BLP_status = {} : BIBA_status = {}".format(replay[1], replay[2]))
        elif replay[1] == "write" and replay[2] == cmd[1]:
            
            if replay[0] == "-1":
                print("ERROR: Authentication failing")

            elif replay[0] == "-2":  
                print("ERROR: There is no such file for writing")


    if cmd[0] == "read" and len(cmd) == 2:  
        if send_command(s, key, command + '\n' + AouthCode) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split("\n",1)
        content = replay[1]
        replay = replay[0].split()
        if replay[3] == "read" and replay[4] == cmd[1]:

            if replay[0] == "1":
                print("File content :\n{}".format(content))

            elif replay[0] == "0" and replay[1] == "1" and replay[2] == "1":
                print("ERROR: Reading file was not successfull try again")

            else:
                print("ERROR: Reading file was not successfull : BLP_status = {} : BIBA_status = {}".format(replay[1], replay[2]))
        elif replay[1] == "read" and replay[2] == cmd[1]:
            
            if replay[0] == "-1":
                print("ERROR: Authentication failing")

            elif replay[0] == "-2":  
                print("ERROR: There is no such file for reading")

    if cmd[0] == "get" and len(cmd) == 2:
        if send_command(s, key, command + "\n" + AouthCode) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split("\n",1)
        content = replay[1]
        replay = replay[0].split()
        
        if replay[1] == "get" and replay[2] == cmd[1]:
            if replay[0] == "1":
                f = open(cmd[1], "w")
                if f == 0:
                    print("ERROR: File is not created on your system")
                else:
                    f.write(content)
                    print("Getting file was successful")
                f.close()

            elif replay[0] == "0":  
                print("ERROR: Getting file was not successfull (permission denied)")

            elif replay[0] == "-1":
                print("ERROR: Authentication failing")
            
            elif replay[0] == "-2":  
                print("ERROR: There is no such file for getting")

    if cmd[0] == "put" and len(cmd) == 5:
        if (os.path.isfile('./' + cmd[1])):
            f = open(cmd[1], "r")
            if f == 0:
                print("ERROR: File could not be opened or file does not exist on your system")
            else:
                content = f.read()
                f.close()
                if send_command(s, key, command + "\n" + content + "\n" + AouthCode) == 0:
                    return 0
                replay = recieve_replay(s, key)
                if replay == None:
                    return 0
                replay = replay.split()
                if replay[1] == "put" and replay[2] == cmd[1]:
                    if replay[0] == "0":
                        print("ERROR: Putting file was not successfull try again")

                    elif replay[0] == "1":
                        print("Putting file was successfull")

                    elif replay[0] == "-1":
                        print("ERROR: Authentication failing")

                    elif replay[0] == "-2":  
                        print("ERROR: There is such file on server")
        else:
            print("ERROR: There is not such file on your system")

    if cmd[0] == "list" and len(cmd) == 1:
        
        if send_command(s, key, command + '\n' + AouthCode) == 0:
            return 0        
        msg = "list"
        replay = recieve_replay(s, key)
        if replay == None:
            return 0
        print(replay)
        replay = replay.split("\n",1)
        content = replay[1]
        replay = replay[0].split()
        if replay[1] == "read" and replay[4] == cmd[1]:

            if replay[0] == "1":
                print("Files list :\n{}".format(content))

            elif replay[0] == "0":
                print("ERROR: Listing files was not successfull try again")

            if replay[0] == "-1":
                print("ERROR: Authentication failing")

    if cmd[0] == "logout" and len(cmd) == 1:

        if send_command(s, key, command + '\n' + AouthCode) == 0:
            return 0
        replay = recieve_replay(s, key)
        
        if replay == None:
            return 0
        replay = replay.split()
        if replay[1] == "logout":
            
            if replay[0] == "0":
                print("ERROR: Logout was not successfull try again")

            elif replay[0] == "1":
                print("Logout was successfull")
                AouthCode = '0'

            elif replay[0] == "-1":
                print("ERROR: Authentication failing")


    
    


In [ ]:


def register(client_sock, key): # Returns 1 if successfully registered, otherwise 0
    
    os.system('cls')
    input_range = ['1', '2', '3', '4']
    while True:
        username = input('Please enter a Username : ')
        if username.count(' ') == 0 and username.count('\\') == 0: # backslash(like \n, \t,...) and spaces may be splitted in commands
            break
        else:
            print('Username can not contain space or backslash.')
    os.system('cls')
    while True:
        password = input('Please enter a Password : ')
        if password.count(' ') == 0 and password.count('\\') == 0:
            break
        else:
            print('Password can not contain space or backslash.')
    os.system('cls')
    while True:
        conf_label = input('Please choose your Confidentiality level :\n4.‫‪Top Secret‬‬\n3.Secret\n2.Confidential\n1.Unclassified\n')
        if conf_label in input_range:
            break
    os.system('cls')
    while True:
        integrity_label = input('Please choose your Integrity level :\n4.‫‪Very Trusted‬‬\n3.Trusted‬‬\n2.SlightlyTrusted\n1.Untrusted‬‬\n')
        if integrity_label in input_range:
            break
    os.system('cls')
    command = 'register '+ username +' '+ password +' '+ conf_label +' '+ integrity_label
    if proccess_command(client_sock, key, command) == 1:
        input('Press enter to continue...\n')
        return 1
    input('Press enter to continue...\n')
    return 0

def login(client_sock, key):

    os.system('cls')
    while True:
        username = input('Please enter Username : ')
        if username.count(' ') == 0 and username.count('\\') == 0:
            break
        else:
            print('Username can not contain space or backslash.')
    os.system('cls')
    while True:
        password = input('Please enter Password : ')
        if password.count(' ') == 0 and password.count('\\') == 0:
            break
        else:
            print('Password can not contain space or backslash.')
    os.system('cls')
    while True:
        if random_captcha() == 1:
            print('Correct answer')
            break
        print('Wrong answer.Try again')
    os.system('cls')
    command = 'login '+ username +' '+ password
    if proccess_command(client_sock, key, command) == 1:
        input('Press enter to continue...\n')
        return 1
    input('Press enter to continue...\n')
    return 0

def put(client_sock, key):

    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    os.system('cls')
    input_range = ['1', '2', '3', '4']
    
    #conflabel_range = ['TopSecret', 'Secret', 'Confidential', 'Unclassified']
    #intlabel_range = ['VeryTrusted', 'Trusted', 'Slightly Trusted', 'UnTrusted']
    
    while True:
        conf_label = input('Please choose file Confidentiality level :\n4.‫‪Top Secret‬‬\n3.Secret\n2.Confidential\n1.Unclassified\n')
        if conf_label in input_range:
            break
    os.system('cls')
    while True:
        integrity_label = input('Please choose file Integrity level :\n4.‫‪Very Trusted‬‬\n3.Trusted‬‬\n2.SlightlyTrusted\n1.Untrusted‬‬\n')
        if integrity_label in input_range:
            break
    os.system('cls')
    access_modes = input('Please choose access modes(2.BLP, 3.BIBA, 5.DAC‬‬)\n')
    access_modes = [char for char in access_modes]
    cond = 1
    for i in access_modes:
        cond = cond * int(i)
   # print(cond)
    os.system('cls')
    command = 'put '+ filename +' '+ conf_label +' '+ integrity_label + ' ' + str(cond)
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')
    
    
def get(client_sock, key):

    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    os.system('cls')
    command = 'get '+ filename
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')

def read(client_sock, key):
    
    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    os.system('cls')
    command = 'read '+ filename
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')

def list_files(client_sock, key):
    
    os.system('cls')
    command = 'list'
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')

def write(client_sock, key):

    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    
    print('Please input the content. Type "end of writing" to finish')
    lines = []
    while True:
        line = input()
        if line != 'end':
            lines.append(line)
        else:
            break
    text = '\n'.join(lines)
    command = 'write '+ filename + '\n' + text
    input('Press enter to continue...\n')
    os.system('cls')
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')

def grant(client_sock, key):

    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    os.system('cls')
    input_range = ['1', '2']
    #conflabel_range = ['TopSecret', 'Secret', 'Confidential', 'Unclassified']
    #intlabel_range = ['VeryTrusted', 'Trusted', 'Slightly Trusted', 'UnTrusted']
    while True:
        username = input('plase enter the username which you want to grant access :')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('username can not contain space or backslash.')
    os.system('cls')
    while True:
        access_type = input('Please choose among the following numbers :\n1.write access\n2.read access\n')
        if access_type in input_range:
            break
    os.system('cls')
    if access_type == 1:
        command = 'grant '+ filename +' ' + username + ' r'
    else:
        command = 'grant '+ filename +' ' + username + ' w'
        
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')

def log_out(client_sock, key):
    command = 'logout'
    after_login(client_sock, key, command)
    

def revoce(client_sock, key):

    os.system('cls')
    while True:
        filename = input('Please enter the File name : ')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('Filename can not contain space or backslash.')
    os.system('cls')
    input_range = ['1', '2']
    #conflabel_range = ['TopSecret', 'Secret', 'Confidential', 'Unclassified']
    #intlabel_range = ['VeryTrusted', 'Trusted', 'Slightly Trusted', 'UnTrusted']
    while True:
        username = input('plase enter the username which you want to grant access :')
        if filename.count(' ') == 0 and filename.count('\\') == 0:
            break
        else:
            print('username can not contain space or backslash.')
    os.system('cls')
    while True:
        access_type = input('Please choose among the following numbers :\n1.write access\n2.read access\n')
        if access_type in input_range:
            break
    os.system('cls')
    if access_type == 1:
        command = 'revoce '+ filename +' ' + username + ' r'
    else:
        command = 'revoce '+ filename +' ' + username + ' w'
        
    after_login(client_sock, key, command)
    input('Press enter to continue...\n')


def first_menu(client_sock, key):

    option = '0'
    while option != '3':
        os.system('clear' or 'cls')
        option = input('Please choose among the following numbers :\n1.Register\n2.Login\n3.Exit\n')
        if option == '1':
            return register(client_sock, key)
        elif option == '2':
            return login(client_sock, key)
    return -1 #To exit the program


def second_menu(client_sock, key):
    
    option = '0'
    while option != '6':
        os.system('cls')
        option = input('Please choose among the following numbers :\n1.Put a file\n2.Get a file\n'
        '3.Read a file\n4.Write in a file\n5.List files\n6.Log Out\n7.Grant Access\n8.Revoce Access\n')
        if option == '1':
            put(client_sock, key)
        elif option == '2':
            get(client_sock, key)
        elif option == '3':
            read(client_sock, key)
        elif option == '4':
            write(client_sock, key)
        elif option == '5':
            list_files(client_sock, key) 
        elif option == '6':
            log_out(client_sock, key)
        elif option == '7':
            grant(client_sock, key)
        elif option == '8':
            revoce(client_sock, key)


def random_captcha():

    img = ImageCaptcha()
    content = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(4))
    image = img.generate_image(content)
    image.show()

    if input('Please enter captcha and close the picture: ').lower() == content :
        return 1
    return 0
   # return 1
        

if __name__ == "__main__":
    
    client_sock = connection_setup()
    key = key_exchange(client_sock)
    input('Press enter to continue...\n')
    while client_sock != None and key != None:
        while AouthCode == '0':
            first_menu(client_sock, key)
        while AouthCode != '0':
            second_menu(client_sock, key)
    client_sock.shutdown()
    client_sock.close()
    os.system('cls')
    print('Goodbye...')
    #X123_bbsara_1289VVV

Connecting to server ... 
Key exchanged successfully.



### 